In [1]:
import ipywidgets as widgets

In [2]:
DEFAULTS = {
    'size': 640,
    'heading': 140,
    'pitch': 10,
    'fov': 50
}

In [3]:
output = widgets.Output()
lbl_style = {'description_width': '200px'}
layout = widgets.Layout(width='600px')
location = widgets.Text(value='33rd & Loch Raven Baltimore MD',
                        description='Location',
                        layout=layout,
                        style=lbl_style)
size = widgets.IntSlider(value=DEFAULTS['size'],
                         min=100,
                         max=1024,
                         layout=layout,
                         style=lbl_style,
                         description='Image size')
heading = widgets.IntSlider(value=DEFAULTS['heading'],
                            min=0,
                            max=360,
                            layout=layout,
                            style=lbl_style,
                            description='Heading (rotation)')
pitch = widgets.IntSlider(value=DEFAULTS['pitch'],
                          min=0,
                          max=40,
                          layout=layout,
                          style=lbl_style,
                          description='Pitch (tilt)')
fov = widgets.IntSlider(value=DEFAULTS['fov'],
                        min=10,
                        max=120,
                        layout=layout,
                        style=lbl_style,
                        description='Field of view (zoom)')

button = widgets.Button(description='Get image')

display(location, size, heading, pitch, fov, button, output)


def button_click(b):
    output.clear_output()
    with output:
        txt = f'location: {location.value}'
        print(txt)
    return txt

sv_img = button.on_click(button_click)


Text(value='33rd & Loch Raven Baltimore MD', description='Location', layout=Layout(width='600px'), style=TextS…

IntSlider(value=640, description='Image size', layout=Layout(width='600px'), max=1024, min=100, style=SliderSt…

IntSlider(value=140, description='Heading (rotation)', layout=Layout(width='600px'), max=360, style=SliderStyl…

IntSlider(value=10, description='Pitch (tilt)', layout=Layout(width='600px'), max=40, style=SliderStyle(descri…

IntSlider(value=50, description='Field of view (zoom)', layout=Layout(width='600px'), max=120, min=10, style=S…

Button(description='Get image', style=ButtonStyle())

Output()